In [2]:
import pandas as pd
import pickle
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm

In [3]:
data_folder = Path('../data/')
pickle_folder = data_folder / 'pickles'
imdb_folder = data_folder / 'IMDB'

In [323]:
with open(pickle_folder / 'movies.p', 'rb') as f:
    movies = pickle.load(f)
movies.year = movies.release_date.dt.year
movies.revenue.fillna(0.0, inplace=True)

In [324]:
movies

,wiki_id,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,year
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001.0
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,0.0,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",2000.0
2,28463795,/m/0crgdbh,Brun bitter,1988-01-01,0.0,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988.0
3,9363483,/m/0285_cd,White Of The Eye,1987-01-01,0.0,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",1987.0
4,261236,/m/01mrr1,A Woman in Flames,1983-01-01,0.0,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",1983.0
...,...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,0.0,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",2011.0
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,0.0,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",2011.0
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,0.0,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",1972.0
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,0.0,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",1992.0


In [325]:
titles = pd.read_csv(imdb_folder / 'title.basics.tsv', sep='\t')
titles.rename({'genres': 'imdb_genres'}, inplace=True)
titles = titles.replace('\\N', None)
#have some corrupted rows, going to drop them
corrupted = titles[titles['runtimeMinutes'].str.isdigit() == False]
display(corrupted)
titles.drop(corrupted.index, inplace=True)

for col in ['isAdult', 'startYear', 'endYear', 'runtimeMinutes']:
    titles[col] = titles[col].apply(pd.to_numeric)

titles.head()

/var/folders/9t/5ml_m_x53vs0ydn7wx7l76l80000gn/T/ipykernel_99514/1673938310.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles = pd.read_csv(imdb_folder / 'title.basics.tsv', sep='\t')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1097679,tt10233364,tvEpisode,Rolling in the Deep Dish\tRolling in the Deep ...,0,2019,None,None,Reality-TV,NaN
1508829,tt10970874,tvEpisode,Die Bauhaus-Stadt Tel Aviv - Vorbild für die M...,0,2019,None,None,Talk-Show,NaN
1897500,tt11670006,tvEpisode,...ein angenehmer Unbequemer...\t...ein angene...,0,1981,None,None,Documentary,NaN
2008573,tt11868642,tvEpisode,GGN Heavyweight Championship Lungs With Mike T...,0,2020,None,None,Talk-Show,NaN
2163214,tt12149332,tvEpisode,Jeopardy! College Championship Semifinal Game ...,0,2020,None,None,Game-Show,NaN
2308653,tt12415330,tvEpisode,Anthony Davis High Brow Tank\tAnthony Davis Hi...,0,2017,None,None,Reality-TV,NaN
2999699,tt13704268,tvEpisode,Bay of the Triffids/Doctor of Doom\tBay of the...,0,None,None,None,"Animation,Comedy,Family",NaN
6675032,tt3984412,tvEpisode,"I'm Not Going to Come Last, I'm Just Going to ...",0,2014,None,None,"Game-Show,Reality-TV",NaN
9311274,tt9822816,tvEpisode,Zwischen Vertuschung und Aufklärung - Missbrau...,0,2019,None,None,Talk-Show,NaN
9351124,tt9909210,tvEpisode,Politik und/oder Moral - Wie weit geht das Ver...,0,2005,None,None,Talk-Show,NaN


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,NaN,4.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1.0,"Comedy,Short"


In [338]:
grouped_title = titles.groupby(['primaryTitle', 'startYear'])
print(len(titles), len(grouped_title))

9354663 5502992


In [339]:
# groups of size 1 (unique title + start year)
titles_deduplicated = grouped_title.filter(lambda x: x['tconst'].count() == 1)

In [340]:
len(titles_deduplicated)

4892448

In [328]:
# groups of size >1 (potential duplicates)
titles_with_duplicates =  grouped_title.filter(lambda x: x['tconst'].count() > 1)
grouped_duplicates = titles_with_duplicates.groupby(['primaryTitle', 'startYear'])

In [329]:
print(len(titles_deduplicated), len(grouped_duplicates))

4892448 366641


Let's have a look at the duplicates, we can see that the types of duplicates we encounter include:
- tv show episodes (in CMU the series are represented as a single (with a really large duration) data point)
- multiple occurances with non-contradicting data (e.g one example is "fuller" than the other)
- multiple occurances with contradicting data (e.g movies with the same name and movie creation but different duration, plausible)

In [31]:
titles_with_duplicates.sort_values('primaryTitle')

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
5998644,tt2399574,short,#,#,0,2012.0,NaN,15.0,"Comedy,Short"
7903681,tt6748908,short,#,#,0,2012.0,NaN,8.0,"Animation,Short"
4759283,tt1813109,video,#1,#1,0,2010.0,NaN,3.0,"Animation,Drama,Short"
5269884,tt2057889,short,#1,#1,0,2010.0,NaN,12.0,"Comedy,Short"
4288335,tt1611790,video,#1,#1,0,2010.0,NaN,12.0,"Comedy,Drama,News"
...,...,...,...,...,...,...,...,...,...
2664560,tt13086812,tvEpisode,Þáttur 5,Þáttur 5,0,2020.0,NaN,NaN,Comedy
2664573,tt13086834,tvEpisode,Þáttur 6,Þáttur 6,0,2020.0,NaN,NaN,Comedy
3909913,tt15372920,tvEpisode,Þáttur 6,Þáttur 6,0,2021.0,NaN,NaN,Reality-TV
3909879,tt15372864,tvEpisode,Þáttur 6,Þáttur 6,0,2020.0,NaN,NaN,Reality-TV


We would run the deduplication procedure as follows:
For every datapoint which has the same title and production year we keep only the fullest of the non-contradicting examples (when checking for contradiction we consider every value but the genres, since they are highly flexible)

In [32]:
import multiprocessing
from deduplicate_helpers import deduplicate_group
N_THREADS = 16


removed_duplicates = pd.DataFrame(columns=titles.columns)

pool = multiprocessing.Pool(N_THREADS)
deduplicated_groups = [removed_duplicates]

i = 0
n = len(grouped_duplicates)
for processed_group in tqdm(pool.imap_unordered(deduplicate_group, grouped_duplicates)):
    deduplicated_groups.append(processed_group)
    if i % 25000 == 0: 
        print(f'{i} groupes ({(i / n) * 100:.1f}%) processed')
    i +=1
removed_duplicates = pd.concat(deduplicated_groups)

0it [00:00, ?it/s]

0 groupes (0.0%) processed
10000 groupes (2.7%) processed
20000 groupes (5.5%) processed
30000 groupes (8.2%) processed
40000 groupes (10.9%) processed
50000 groupes (13.6%) processed
60000 groupes (16.4%) processed
70000 groupes (19.1%) processed
80000 groupes (21.8%) processed
90000 groupes (24.5%) processed
100000 groupes (27.3%) processed
110000 groupes (30.0%) processed
120000 groupes (32.7%) processed
130000 groupes (35.5%) processed
140000 groupes (38.2%) processed
150000 groupes (40.9%) processed
160000 groupes (43.6%) processed
170000 groupes (46.4%) processed
180000 groupes (49.1%) processed
190000 groupes (51.8%) processed
200000 groupes (54.5%) processed
210000 groupes (57.3%) processed
220000 groupes (60.0%) processed
230000 groupes (62.7%) processed
240000 groupes (65.5%) processed
250000 groupes (68.2%) processed
260000 groupes (70.9%) processed
270000 groupes (73.6%) processed
280000 groupes (76.4%) processed
290000 groupes (79.1%) processed
300000 groupes (81.8%) proce

In [35]:
titles_deduplicated = pd.concat([titles_deduplicated, removed_duplicates])

In [36]:
with open(pickle_folder / "deduplicated_titles_temp.p", "wb") as f:
    pickle.dump(titles_deduplicated, f)

In [341]:
with open(pickle_folder / 'deduplicated_titles_temp.p', 'rb') as f:
    titles_deduplicated = pickle.load(f)

In [33]:
print(len(removed_duplicates), len(titles_with_duplicates))

765254 3207314


A proxi to the final share of matched movies (not exact, bc we will be matching on movie release year and duration as well)

In [37]:
cmu_movie_names = set(movies.name)
imdb_movie_names_orginal = set(titles_deduplicated.originalTitle)
imdb_movie_names_primary = set(titles_deduplicated.primaryTitle)

intersect_orig = cmu_movie_names.intersection(imdb_movie_names_orginal)
intersect_primary = cmu_movie_names.intersection(imdb_movie_names_primary)
intersect_imdb = cmu_movie_names.intersection(imdb_movie_names_primary.union(imdb_movie_names_orginal))

print(f"Intersection with original titles: {len(intersect_orig)} ({len(intersect_orig) / len(cmu_movie_names) * 100:.2f}%)")
print(f"Intersection with primary titles: {len(intersect_primary)} ({len(intersect_primary) / len(cmu_movie_names) * 100:.2f}%)")
print(f"Intersection with imdb titles: {len(intersect_imdb)} ({len(intersect_imdb) / len(cmu_movie_names) * 100:.2f}%)")

Intersection with original titles: 57085 (75.63%)
Intersection with primary titles: 59919 (79.39%)
Intersection with imdb titles: 62121 (82.30%)


In [38]:
movies[movies.name == 'Les Misérables']

,wiki_id,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,year
11234,16679395,/m/03ym8tz,Les Misérables,1958-03-12,NaN,102.0,{},"{""/m/0f8l9c"": ""France"", ""/m/03f2w"": ""German De...","{""/m/07s9rl0"": ""Drama""}",1958.0
15056,22554371,/m/05zt4bn,Les Misérables,1978-12-27,NaN,123.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/04xvlr"": ""Period piece"", ""/m/07s9rl0"": ""D...",1978.0
21050,6243001,/m/0fyjvp,Les Misérables,1982-10-20,NaN,NaN,"{""/m/064_8sq"": ""French Language""}","{""/m/0f8l9c"": ""France""}","{""/m/07s9rl0"": ""Drama""}",1982.0
22929,33029735,/m/0h51_mt,Les Misérables,2012-12-14,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/04t36"": ""Musical"", ""/m/07s9rl0"": ""Drama""}",2012.0
23611,1743227,/m/05sjt9,Les Misérables,1998-05-01,14096322.0,159.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/04p5fxn"": ""Christian film"", ""/m/03g3w"": ""...",1998.0
25449,22810161,/m/06408_y,Les Misérables,1925-12-31,NaN,359.0,"{""/m/06ppq"": ""Silent film""}","{""/m/0f8l9c"": ""France""}","{""/m/06ppq"": ""Silent film"", ""/m/01g6gs"": ""Blac...",1925.0
27085,22810098,/m/063zppt,Les Misérables,1909-11-27,NaN,NaN,{},"{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",1909.0
42028,1261542,/m/04n2_y,Les Misérables,1995-03-22,NaN,174.0,"{""/m/064_8sq"": ""French Language"", ""/m/04306rv""...","{""/m/0f8l9c"": ""France""}","{""/m/07s9rl0"": ""Drama"", ""/m/03q4nz"": ""World ci...",1995.0
45841,21958457,/m/05p7hqd,Les Misérables,1948-01-21,NaN,188.0,"{""/m/02bjrlw"": ""Italian Language""}","{""/m/03rjj"": ""Italy""}","{""/m/07s9rl0"": ""Drama""}",1948.0
48256,310436,/m/01t39x,Les Misérables,1952-08-14,1100000.0,105.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04xvlr"": ""Period piece"", ""/m/07s9rl0"": ""D...",1952.0


In [342]:
print(f"total movies: {len(movies)}")
movies_imdb = pd.merge(movies, titles_deduplicated.add_suffix('_imdb_primary'), 
                       how="left", left_on=["name", "year"], right_on=["primaryTitle_imdb_primary", "startYear_imdb_primary"])
print(f"Share of matched movies: {(1 - (movies_imdb.primaryTitle_imdb_primary.isna()).mean()):.2f}, total movies: {len(movies_imdb)}")

diff_titles_idx = titles_deduplicated['primaryTitle'] != titles_deduplicated['originalTitle']
movies_imdb = pd.merge(movies_imdb, titles_deduplicated[diff_titles_idx].add_suffix('_imdb_original'), 
                       how="left", left_on=["name", "year"], right_on=["originalTitle_imdb_original", "startYear_imdb_original"])
print(f"Share of matched movies: {(1 - (movies_imdb.primaryTitle_imdb_primary.isna() & movies_imdb.primaryTitle_imdb_original.isna()).mean()):.2f}, total movies: {len(movies_imdb)}")

total movies: 81741
Share of matched movies: 0.68, total movies: 88225
Share of matched movies: 0.71, total movies: 88231


In [343]:
for col in titles.columns:
    filler = 0 if titles[col].dtype in (float, int) else ''
    movies_imdb[col] = (movies_imdb[f'{col}_imdb_primary'].fillna(filler) + \
                                      movies_imdb[f'{col}_imdb_original'].fillna(filler)).copy()

    # so we don't confuse the missing data with the existing but non-informative 0 / ''
    na = np.nan if titles[col].dtype in (float, int) else ''
    both_na_cols = movies_imdb[f'{col}_imdb_primary'].isna() & movies_imdb[f'{col}_imdb_original'].isna()
    movies_imdb.loc[both_na_cols, col] = na
    movies_imdb.drop(columns=[f'{col}_imdb_primary', f'{col}_imdb_original'], inplace=True)
movies_imdb.drop(columns=['year', 'name'])
movies_imdb['endYear'].fillna(0., inplace=True)
movies_imdb.head()

,wiki_id,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","Action,Horror,Sci-Fi",2001.0,tt0228333,movie,Ghosts of Mars,Ghosts of Mars,0.0,2001.0,0.0,98.0
1,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","Documentary,Short",2001.0,tt0594845,tvEpisode,Ghosts of Mars,Ghosts of Mars,0.0,2001.0,0.0,NaN
2,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,0.0,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",,2000.0,,,,,NaN,NaN,0.0,NaN
3,28463795,/m/0crgdbh,Brun bitter,1988-01-01,0.0,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","Crime,Drama",1988.0,tt0094806,movie,Brun bitter,Brun bitter,0.0,1988.0,0.0,83.0
4,9363483,/m/0285_cd,White Of The Eye,1987-01-01,0.0,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}",,1987.0,,,,,NaN,NaN,0.0,NaN


### Filtering wrong mappings

In [344]:
# checking for contradicting imdb ids
wikidata_to_imdb_id_mapping = pd.read_csv(imdb_folder / 'complete_wikiID_imdbID_mapping.csv') 
wikidata_to_imdb_id_mapping.drop(columns=['Unnamed: 0'], inplace=True)

with open(pickle_folder / 'freebase_to_wikidata.p', 'rb') as f:
    freebase_to_wikidata_mapping = pickle.load(f)

freebase_to_imdb_mapping = wikidata_to_imdb_id_mapping.merge(freebase_to_wikidata_mapping, on='wikidata_id', how='inner')
display(freebase_to_imdb_mapping.head())

movies_imdb = movies_imdb.merge(freebase_to_imdb_mapping, on='freebase_id', how='left')
display(movies_imdb.head())

,wikidata_id,IMDB_id,wikidata_page,freebase_id
0,Q368,nm0684496,http://www.wikidata.org/entity/Q368,/m/014d3
1,Q501,nm0061667,http://www.wikidata.org/entity/Q501,/m/01rgr
2,Q595,tt1675434,http://www.wikidata.org/entity/Q595,/m/0hgqwpj
3,Q633,nm0949918,http://www.wikidata.org/entity/Q633,/m/0m2l9
4,Q747,nm0180134,http://www.wikidata.org/entity/Q747,/m/0fyh4


,wiki_id,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,year,...,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,wikidata_id,IMDB_id,wikidata_page
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","Action,Horror,Sci-Fi",2001.0,...,movie,Ghosts of Mars,Ghosts of Mars,0.0,2001.0,0.0,98.0,Q261700,tt0228333,http://www.wikidata.org/entity/Q261700
1,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","Documentary,Short",2001.0,...,tvEpisode,Ghosts of Mars,Ghosts of Mars,0.0,2001.0,0.0,NaN,Q261700,tt0228333,http://www.wikidata.org/entity/Q261700
2,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,0.0,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",,2000.0,...,,,,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,28463795,/m/0crgdbh,Brun bitter,1988-01-01,0.0,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","Crime,Drama",1988.0,...,movie,Brun bitter,Brun bitter,0.0,1988.0,0.0,83.0,Q4978832,tt0094806,http://www.wikidata.org/entity/Q4978832
4,9363483,/m/0285_cd,White Of The Eye,1987-01-01,0.0,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}",,1987.0,...,,,,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [345]:
def filter_imdb_by_id(df):
    # both non none but not equal => contradiction
    is_contradiction = (df['IMDB_id'] != df['tconst']) & (~df['IMDB_id'].isna()) & (~df['tconst'].isna())
    return is_contradiction

contradicting_imdb_idx = filter_imdb_by_id(movies_imdb)
print(f'filtering out: {sum(contradicting_imdb_idx)} elements')
movies_imdb = movies_imdb[~contradicting_imdb_idx]
movies_imdb.drop(columns=['wikidata_id', 'IMDB_id', 'wikidata_page'], inplace=True)

filtering out: 17023 elements


In [346]:
# filtering by runtime 
def filter_runtime_idx(df):
    return abs(df['runtime'] - df['runtimeMinutes']) / df['runtime'] > 0.2

wrong_runtime_idx = filter_runtime_idx(movies_imdb)
print(f'filtering out: {sum(wrong_runtime_idx)} elements')
movies_imdb = movies_imdb[~wrong_runtime_idx]
movies_imdb['runtime'] = np.maximum(movies_imdb['runtime'].fillna(0), movies_imdb['runtimeMinutes'].fillna(0))
movies_imdb.drop(columns=['runtimeMinutes'], inplace=True)

filtering out: 2153 elements


In [347]:
joined_imdb_data = movies_imdb[movies_imdb.tconst != '']
print(len(joined_imdb_data))
joined_imdb_data.head()

with open(pickle_folder / "movies_with_imdb_ids.p", "wb") as f:
    pickle.dump(joined_imdb_data, f)

54929


In [4]:
with open(pickle_folder / "movies_with_imdb_ids.p", "rb") as f:
    joined_imdb_data = pickle.load(f)

## Merging with crew data

In [5]:
crew = pd.read_csv(imdb_folder / 'title.crew.tsv', sep='\t')
for col in ['directors', 'writers']:
    crew[col] = crew[col].str.replace(',', ';')
crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [6]:
names = pd.read_csv(imdb_folder / 'name.basics.tsv', sep='\t')
for col in ['primaryProfession', 'knownForTitles']:
    names[col] = names[col].str.replace(',', ';')
names.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,soundtrack;actor;miscellaneous,tt0031983;tt0072308;tt0050419;tt0053137
1,nm0000002,Lauren Bacall,1924,2014,actress;soundtrack,tt0071877;tt0117057;tt0037382;tt0038355
2,nm0000003,Brigitte Bardot,1934,\N,actress;soundtrack;music_department,tt0049189;tt0054452;tt0056404;tt0057345
3,nm0000004,John Belushi,1949,1982,actor;soundtrack;writer,tt0078723;tt0077975;tt0080455;tt0072562
4,nm0000005,Ingmar Bergman,1918,2007,writer;director;actor,tt0083922;tt0050976;tt0060827;tt0050986


In [7]:
imdb_data_with_crew = joined_imdb_data.merge(crew, on='tconst', how='inner')
for col in ['directors', 'writers']:
    imdb_data_with_crew[col] = imdb_data_with_crew[col].fillna('\\N')
print(len(imdb_data_with_crew), len(joined_imdb_data))

54884 54929


In [14]:
imdb_data_with_crew.columns

Index(['wiki_id', 'freebase_id', 'name', 'release_date', 'revenue', 'runtime',
       'languages', 'countries', 'genres', 'year', 'tconst', 'titleType',
       'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear',
       'directors', 'writers'],
      dtype='object')

In [8]:
imdb_data_with_crew.isna().describe()

,wiki_id,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,directors,writers
count,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884
unique,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884,54884


In [9]:
def explode_merge_group(left_df, right_df, left_column, right_column):
    left_copy_df = left_df.copy()
    grouping_cols = list(left_df.columns.difference([left_column]))
    left_copy_df[left_column] = left_copy_df[left_column].str.split(';')
    exploded = left_copy_df.explode(left_column)
    exploded = exploded.merge(right_df.add_prefix(f'{left_column}_'), 
                              left_on=left_column, right_on=f'{left_column}_{right_column}',
                              how='inner')
    agg_cols = list(exploded.columns.difference(grouping_cols))
    grouped = exploded.groupby(by=grouping_cols)
    unexploded = grouped.agg({agg_col: list for agg_col in agg_cols})
    return unexploded

In [10]:
movies_with_directors = explode_merge_group(imdb_data_with_crew, names, 'directors', 'nconst').reset_index()

In [11]:
movies_with_directors

,countries,endYear,freebase_id,genres,isAdult,languages,name,originalTitle,primaryTitle,release_date,...,wiki_id,writers,year,directors,directors_birthYear,directors_deathYear,directors_knownForTitles,directors_nconst,directors_primaryName,directors_primaryProfession
0,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",0.0,/m/076x850,"Comedy,Drama,Romance",0.0,"{""/m/02h40lc"": ""English Language""}",Twenty Four Seven,TwentyFourSeven,Twenty Four Seven,1997-08-31,...,23904795,nm0292211;nm0276349,1997.0,[nm0276349],[1972],[\N],[tt0419677;tt0480025;tt0120391;tt2528254],[nm0276349],[Shane Meadows],[director;writer;producer]
1,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",0.0,/m/04f_y7,"Biography,Drama,Romance",0.0,"{""/m/02h40lc"": ""English Language""}",Wilde,Wilde,Wilde,1997-01-01,...,1191380,nm0593483;nm0255264,1997.0,[nm0317981],[1960],[\N],[tt0120514;tt0294594;tt0111454;tt0087300],[nm0317981],[Brian Gilbert],[director;actor;writer]
2,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",0.0,/m/0fbp6n,Drama,0.0,"{""/m/02h40lc"": ""English Language""}",The Snapper,The Snapper,The Snapper,1993-08-06,...,5882959,nm0236486,1993.0,[nm0001241],[1941],[\N],[tt0094947;tt0301199;tt2431286;tt0436697],[nm0001241],[Stephen Frears],[director;producer;actor]
3,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",0.0,/m/03ljq9,"Crime,Thriller",0.0,"{""/m/02h40lc"": ""English Language""}",Shallow Grave,Shallow Grave,Shallow Grave,1994-01-01,...,883437,nm0388076,1994.0,[nm0000965],[1956],[\N],[tt0111149;tt1542344;tt0289043;tt1010048],[nm0000965],[Danny Boyle],[director;producer;writer]
4,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",0.0,/m/080lv9k,Drama,0.0,"{""/m/02h40lc"": ""English Language""}",Under the Skin,Under the Skin,Under the Skin,1997-08-29,...,24517862,nm0012119,1997.0,[nm0012119],[1948],[\N],[tt0208144;tt0329926;tt0124185;tt0264090],[nm0012119],[Carine Adler],[director;writer;script_department]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54002,{},2009.0,/m/09gkj4m,"Biography,Drama,History",0.0,"{""/m/032f6"": ""Persian Language""}",Yousuf e Payambar,Yousuf e Payambar,Yousuf e Payambar,2008-01-01,...,25046162,\N,2008.0,[nm2611865],[1952],[2016],[tt3084132;tt3084140;tt10576218;tt5784052],[nm2611865],[Farajollah Salahshoor],[director;actor;writer]
54003,{},2010.0,/m/03yn08,"Action,Adventure,Animation",0.0,{},Atomic Betty,Atomic Betty,Atomic Betty,2004-01-01,...,1006436,nm0203614;nm0971764;nm0142830;nm0589076;nm1729...,2004.0,"[nm0142830, nm1740283, nm1966844, nm1967766, n...","[\N, \N, 1973, \N, \N, \N, \N, \N, \N, 1957]","[\N, \N, \N, \N, \N, \N, \N, \N, \N, \N]","[tt0324919;tt2170392;tt1577521;tt0424603, tt04...","[nm0142830, nm1740283, nm1966844, nm1967766, n...","[Mauro Casalese, Virginie Jallot, Ridd Sorense...","[director;producer;animation_department, direc..."
54004,{},2010.0,/m/0fqnyhz,Documentary,0.0,{},Chemistry: A Volatile History,Chemistry: A Volatile History,Chemistry: A Volatile History,2010-01-01,...,29952420,\N,2010.0,"[nm1657474, nm2454034, nm1243742]","[\N, \N, \N]","[\N, \N, \N]","[tt2969252;tt1588194;tt6408260;tt3120454, tt15...","[nm1657474, nm2454034, nm1243742]","[Tom Cholmondeley, Ros Homan, Jon Stephens]","[director;producer;miscellaneous, producer;dir..."
54005,{},2012.0,/m/0l8lv85,"Drama,Mystery,Sci-Fi",0.0,{},Coma,Coma,Coma,2012-09-03,...,36712306,nm0177257;nm0572352,2012.0,[nm0004121],[1945],[\N],[tt0185906;tt0384766;tt0096754;tt3230854],[nm0004121],[Mikael Salomon],[director;cinematographer;producer]


In [12]:
# merging them separately, because produced lists are unhashable (can not do groupby by it)
movies_with_writers = explode_merge_group(imdb_data_with_crew, names, 'writers', 'nconst').reset_index()

In [13]:
movies_with_writers 

,countries,directors,endYear,freebase_id,genres,isAdult,languages,name,originalTitle,primaryTitle,...,titleType,wiki_id,year,writers,writers_birthYear,writers_deathYear,writers_knownForTitles,writers_nconst,writers_primaryName,writers_primaryProfession
0,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",nm0276349,0.0,/m/076x850,"Comedy,Drama,Romance",0.0,"{""/m/02h40lc"": ""English Language""}",Twenty Four Seven,TwentyFourSeven,Twenty Four Seven,...,movie,23904795,1997.0,"[nm0292211, nm0276349]","[1973, 1972]","[\N, \N]","[tt0419677;tt0202559;tt0120391;tt1172206, tt04...","[nm0292211, nm0276349]","[Paul Fraser, Shane Meadows]","[writer;director;actor, director;writer;producer]"
1,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",nm0317981,0.0,/m/04f_y7,"Biography,Drama,Romance",0.0,"{""/m/02h40lc"": ""English Language""}",Wilde,Wilde,Wilde,...,movie,1191380,1997.0,"[nm0593483, nm0255264]","[1935, 1918]","[\N, 1987]","[tt0092379;tt0060121;tt1059848;tt0086904, tt21...","[nm0593483, nm0255264]","[Julian Mitchell, Richard Ellmann]","[writer;actor;miscellaneous, writer]"
2,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",nm0001241,0.0,/m/0fbp6n,Drama,0.0,"{""/m/02h40lc"": ""English Language""}",The Snapper,The Snapper,The Snapper,...,tvEpisode,5882959,1993.0,[nm0236486],[1958],[\N],[tt0109769;tt0101605;tt8247470;tt6323304],[nm0236486],[Roddy Doyle],[writer;producer]
3,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",nm0000965,0.0,/m/03ljq9,"Crime,Thriller",0.0,"{""/m/02h40lc"": ""English Language""}",Shallow Grave,Shallow Grave,Shallow Grave,...,movie,883437,1994.0,[nm0388076],[1964],[\N],[tt0111149;tt0117951;tt0163978;tt2763304],[nm0388076],[John Hodge],[writer;actor;producer]
4,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",nm0001402,0.0,/m/0bk6f0,"Adventure,Comedy,Family",0.0,"{""/m/02h40lc"": ""English Language""}",The Wind in the Willows,The Wind in the Willows,Mr. Toad's Wild Ride,...,movie,4120837,1996.0,"[nm0334370, nm0001402]","[1859, 1942]","[1932, 2020]","[tt1600212;tt0046593;tt0051312;tt0041094, tt00...","[nm0334370, nm0001402]","[Kenneth Grahame, Terry Jones]","[writer, writer;actor;producer]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50214,{},nm8833411,0.0,/m/0j7m04b,"Drama,Short",0.0,{},Lyiza,Lyiza,Lyiza,...,short,35181035,2011.0,[nm8833411],[\N],[\N],[tt11297018;tt6620880;tt15101638;tt12615698],[nm8833411],[Marie-Clementine Dusabejambo],[director;writer;producer]
50215,{},nm9280062,0.0,/m/0j9mnt6,"Action,Comedy,Crime",0.0,{},Apmaan Ki Aag,Apmaan Ki Aag,Apmaan Ki Aag,...,movie,35368773,1990.0,"[nm9280062, nm1007381, nm0759785, nm0755991]","[\N, \N, \N, \N]","[\N, \N, \N, 2020]","[tt0482034;tt0360227;tt0343048;tt4616354, tt21...","[nm9280062, nm1007381, nm0759785, nm0755991]","[Sardar Talukdar, Tanveer Khan, Sameer, Anwar ...","[director;writer;actor, writer;director;actor,..."
50216,{},nm9432331,0.0,/m/0j63z3m,"Documentary,Short",0.0,{},Marfil,Marfil,Marfil,...,short,35013748,2011.0,[nm9432331],[\N],[\N],[tt7653860;tt8302492;tt12763158],[nm9432331],[Rubén Monsuy Ndong Andeme],[writer;director;editor]
50217,{},nm9703058,0.0,/m/0hglldd,Romance,0.0,"{""/m/03k50"": ""Hindi Language""}",Pal Pal Dil Ke Ssaat,Pal Pal Dil Ke Ssaat,Pal Pal Dil Ke Ssaat,...,movie,33636196,2009.0,[nm9703057],[\N],[\N],[tt4411786;tt1800369],[nm9703057],[Farauq Malik],[writer;assistant_director]


In [26]:
join_cols = list(imdb_data_with_crew.columns.difference(['directors', 'writers']))
movies_with_crew = movies_with_directors.merge(movies_with_writers, on=join_cols, how='inner')
movies_with_crew.drop(columns=['writers_x', 'directors_y', 'writers_y', 'directors_x'], inplace=True)
movies_with_crew.head()

,countries,endYear,freebase_id,genres,isAdult,languages,name,originalTitle,primaryTitle,release_date,...,directors_knownForTitles,directors_nconst,directors_primaryName,directors_primaryProfession,writers_birthYear,writers_deathYear,writers_knownForTitles,writers_nconst,writers_primaryName,writers_primaryProfession
0,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",0.0,/m/076x850,"Comedy,Drama,Romance",0.0,"{""/m/02h40lc"": ""English Language""}",Twenty Four Seven,TwentyFourSeven,Twenty Four Seven,1997-08-31,...,[tt0419677;tt0480025;tt0120391;tt2528254],[nm0276349],[Shane Meadows],[director;writer;producer],"[1973, 1972]","[\N, \N]","[tt0419677;tt0202559;tt0120391;tt1172206, tt04...","[nm0292211, nm0276349]","[Paul Fraser, Shane Meadows]","[writer;director;actor, director;writer;producer]"
1,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",0.0,/m/04f_y7,"Biography,Drama,Romance",0.0,"{""/m/02h40lc"": ""English Language""}",Wilde,Wilde,Wilde,1997-01-01,...,[tt0120514;tt0294594;tt0111454;tt0087300],[nm0317981],[Brian Gilbert],[director;actor;writer],"[1935, 1918]","[\N, 1987]","[tt0092379;tt0060121;tt1059848;tt0086904, tt21...","[nm0593483, nm0255264]","[Julian Mitchell, Richard Ellmann]","[writer;actor;miscellaneous, writer]"
2,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",0.0,/m/0fbp6n,Drama,0.0,"{""/m/02h40lc"": ""English Language""}",The Snapper,The Snapper,The Snapper,1993-08-06,...,[tt0094947;tt0301199;tt2431286;tt0436697],[nm0001241],[Stephen Frears],[director;producer;actor],[1958],[\N],[tt0109769;tt0101605;tt8247470;tt6323304],[nm0236486],[Roddy Doyle],[writer;producer]
3,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",0.0,/m/03ljq9,"Crime,Thriller",0.0,"{""/m/02h40lc"": ""English Language""}",Shallow Grave,Shallow Grave,Shallow Grave,1994-01-01,...,[tt0111149;tt1542344;tt0289043;tt1010048],[nm0000965],[Danny Boyle],[director;producer;writer],[1964],[\N],[tt0111149;tt0117951;tt0163978;tt2763304],[nm0388076],[John Hodge],[writer;actor;producer]
4,"{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...",0.0,/m/080lv9k,Drama,0.0,"{""/m/02h40lc"": ""English Language""}",Under the Skin,Under the Skin,Under the Skin,1997-08-29,...,[tt0208144;tt0329926;tt0124185;tt0264090],[nm0012119],[Carine Adler],[director;writer;script_department],[1948],[\N],[tt0208144;tt0329926;tt0124185;tt0264090],[nm0012119],[Carine Adler],[director;writer;script_department]


In [28]:
with open(pickle_folder / "movies_with_imdb_crew.p", "wb") as f:
    pickle.dump(movies_with_crew, f)